<a href="https://colab.research.google.com/github/agarnitin86/transformers/blob/main/bert_fine_tune_native_pytorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://huggingface.co/docs/transformers/training#train-with-pytorch-trainer

In [ ]:
!pip install transformers datasets evaluate

In [ ]:
from datasets import Dataset
from datasets import load_dataset

import pandas as pd

In [ ]:
train_path = '.'
train_file_name = 'train_all_tasks.csv'

In [ ]:
## Load dataset from a csv file
df = pd.read_csv(train_file_name)

df['label'] = 1
df.loc[df['label_sexist']=='not sexist', 'lable'] = 0
df = df[['text', 'label']]
dataset = Dataset.from_pandas(df)
dataset

# ## Code to load imdb data set
# imdb = load_dataset("imdb")
# #dataset = load_dataset("csv", data_files={'train':train_path+train_file_name})

In [ ]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

In [ ]:
def preprocess_function(examples):
    return tokenizer(examples["text"], truncation=True, padding=True, pad_to_multiple_of=512)

In [ ]:
tokenized_datasets = dataset.map(preprocess_function, batched=True)

In [ ]:
tokenized_datasets = tokenized_datasets.remove_columns(["text"])
tokenized_datasets = tokenized_datasets.rename_column("label", "labels")
tokenized_datasets.set_format("torch")

In [ ]:
#tokenized_datasets['train'][1]['input_ids']
tokenized_datasets[1]['input_ids']

In [ ]:
# ## For imdb dataset
# small_train_dataset = tokenized_datasets["train"].shuffle(seed=42).select(range(1000))
# small_eval_dataset = tokenized_datasets["test"].shuffle(seed=42).select(range(1000))

shuffled_dataset = tokenized_datasets.shuffle(seed=42)
small_train_dataset = shuffled_dataset.select(range(12000))
small_eval_dataset  = shuffled_dataset.select(range(12001,14000))

In [ ]:
from torch.utils.data import DataLoader

train_dataloader = DataLoader(small_train_dataset, shuffle=True, batch_size=8)
eval_dataloader = DataLoader(small_eval_dataset, batch_size=8)

In [ ]:
from transformers import AutoModelForSequenceClassification
#model = AutoModelForSequenceClassification.from_pretrained("bert-base-cased", num_labels=5)
model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=2)


In [ ]:
from torch.optim import AdamW

optimizer = AdamW(model.parameters(), lr=5e-5)

In [ ]:
from transformers import get_scheduler

num_epochs = 3
num_training_steps = num_epochs * len(train_dataloader)
lr_scheduler = get_scheduler(
    name="linear", optimizer=optimizer, num_warmup_steps=0, num_training_steps=num_training_steps
)

In [ ]:
import torch

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model.to(device)

In [ ]:
for batch in train_dataloader:
    print(batch['input_ids'].shape)
    break
    # for key, val in batch.items():
    #     print(key)
    #     print(val)
    #     break
    # break

In [ ]:
from tqdm.auto import tqdm

progress_bar = tqdm(range(num_training_steps))

model.train()
for epoch in range(num_epochs):
    for batch in train_dataloader:
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        loss = outputs.loss
        print(epoch,loss)
        loss.backward()

        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        progress_bar.update(1)

model.save_pretrained("distilbert-base-uncased-semeval-10")        

In [ ]:
import evaluate

metric = evaluate.load("f1")
model.eval()
for batch in eval_dataloader:
    batch = {k: v.to(device) for k, v in batch.items()}
    with torch.no_grad():
        outputs = model(**batch)

    logits = outputs.logits
    predictions = torch.argmax(logits, dim=-1)
    metric.add_batch(predictions=predictions, references=batch["labels"])

metric.compute()

In [ ]:
small_eval_dataset['input_ids']

In [ ]:
model(small_eval_dataset['input_ids'].to(device))

In [ ]:
sentences = dataset.to_pandas()['text'].tolist()
sentence_encoding = tokenizer(sentences[10], truncation=True, padding=True, return_tensors='pt')
model.to(torch.device("cpu"))
embedings = model(**sentence_encoding.to(torch.device("cpu")))

In [ ]:
outputs['logits']